In [1]:
import json
import os
import time
import pandas as pd
import random
import time
import pickle

import numpy as np
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout

from keras.backend import clear_session
from keras.models import model_from_json
from keras.callbacks import ModelCheckpoint
from keras.callbacks import TensorBoard
import tensorflow as tf
from keras import backend as K
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
K.set_session(session)

Using TensorFlow backend.


In [2]:
with open('clean_hp_comments.json') as json_file:  #first 1mill bb comments
    data = json.load(json_file)

In [3]:

hp_tok = Tokenizer(oov_token='<unk>')
hp_tok.fit_on_texts(data)

In [4]:
len(hp_tok.word_index)

10002

In [5]:
with open('hp-v5-tok.pickle', 'wb') as handle:
    pickle.dump(hp_tok, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [6]:


train_data = data

seqs = hp_tok.texts_to_sequences(train_data)
less_20_seqs = list(filter(lambda x: len(x) <20, seqs))

In [7]:
len(less_20_seqs)

705009

In [8]:
MAXLEN = 19
VOCAB = int(len(hp_tok.word_index))
SEED = 42

sequences = pad_sequences(less_20_seqs, maxlen=MAXLEN, padding='pre')

print('Max Seq Len: %d' % MAXLEN)

sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.15, random_state=SEED)


#y_cat = to_categorical(y, num_classes=vocab_sz) #1-hot encoding 

Max Seq Len: 19


Need to optimize this later, use np arrays instead of python lists


For reference, seqs took 57s to go through 1 mill samples

In [ ]:


#NN definition
model = Sequential()
model.add(Embedding(VOCAB, 300, input_length=MAXLEN-1))
model.add(LSTM(500))



model.add(Dense(VOCAB, activation='softmax'))
model.load_weights('weights/huffpo-v5/baseline-weights.hdf5')
print(model.summary())


Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 18, 300)           3000600   
_________________________________________________________________
lstm_1 (LSTM)                (None, 500)               1602000   
_________________________________________________________________
dense_1 (Dense)              (None, 10002)             5011002   
Total params: 9,613,602
Trainable params: 9,613,602
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam', metrics=['accuracy'])


In [ ]:

model_json = model.to_json()
with open("models/hp-model-v5.json", "w") as json_file:
    json_file.write(model_json)

fpath = "weights/huffpo-v5/best-weights.hdf5" #off of training acc
checkpoint = ModelCheckpoint(fpath, monitor='acc', verbose=2, save_best_only=True, mode='max')


BS = 100
tb = TensorBoard(log_dir="tensorboard-logs/{}".format('huffpo-modelv5-run2'))
callback_lst = [checkpoint, tb]
#steps_per_epoch is num of batches that make up 1 epoch, defaults to size of train set
model.fit(X_train,y_train,batch_size=BS, validation_split=.01, epochs=200, callbacks=callback_lst, verbose=1)



Instructions for updating:
Use tf.cast instead.
Train on 593264 samples, validate on 5993 samples
Epoch 1/200
593264/593264 [==============================] - 159s 267us/step - loss: 3.7635 - acc: 0.3305 - val_loss: 4.8890 - val_acc: 0.2833

Epoch 00001: acc improved from -inf to 0.33052, saving model to weights/huffpo-v5/best-weights.hdf5
Epoch 2/200
593264/593264 [==============================] - 159s 268us/step - loss: 3.3548 - acc: 0.3628 - val_loss: 5.0220 - val_acc: 0.2827

Epoch 00002: acc improved from 0.33052 to 0.36277, saving model to weights/huffpo-v5/best-weights.hdf5
Epoch 3/200
593264/593264 [==============================] - 159s 267us/step - loss: 3.0683 - acc: 0.3987 - val_loss: 5.1428 - val_acc: 0.2703

Epoch 00003: acc improved from 0.36277 to 0.39867, saving model to weights/huffpo-v5/best-weights.hdf5
Epoch 4/200
593264/593264 [==============================] - 158s 266us/step - loss: 2.8372 - acc: 0.4325 - val_loss: 5.2508 - val_acc: 0.2757

Epoch 00004: acc imp